# Classification example demonstrating stacking and mean-encoding

In [1]:
import pandas as pd
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status', 'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']

train = pd.read_csv('adult.data', sep=",\s", header=None, names = column_names, engine = 'python')
test = pd.read_csv('adult.test', sep=",\s", header=None, names = column_names, engine = 'python')

# test['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')

In [2]:
test.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,|1x3 Cross validator,None,NaN,None,NaN,None,None,None,None,None,NaN,NaN,NaN,None,None
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.


In [3]:
test.drop(axis=0, index=0, inplace=True)
test.reset_index(inplace=True, drop=True)
test.income = [int(i == '>50K.') for i in test.income]
train.income = [int(i == '>50K') for i in train.income]
test_label = test.income
test =test.drop('income', axis=1)
train_label = train.income
train = train.drop('income', axis=1)

In [4]:
categorical = [var for var in train.columns if train[var].dtype=='O']
numerical = [var for var in train.columns if train[var].dtype!='O']

In [5]:
!pip install category_encoders

In [6]:
import sys
import os
import numpy as np
sys.path.append(os.path.abspath('../scripts'))
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from validatedstackedmeanencoder import ValidatedStackedMeanEncoder

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler())])

#categorical_transformer = ValidatedStackedMeanEncoder(cols = categorical)
# categorical_transformer = Pipeline(steps=[
#     ('ohe', OneHotEncoder()),
#     ('mean_enc', ValidatedStackedMeanEncoder(cols=categorical))
# ])
     

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical),
        ('mean_enc', ValidatedStackedMeanEncoder(), categorical)
        #('cat', categorical_transformer, categorical),
        ])

In [7]:
import lightgbm
model = lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.1, 
                                n_estimators=500, subsample_for_bin=20000, objective='binary', 
                                subsample=1.0, subsample_freq=0, colsample_bytree=1.0, 
                                n_jobs=- 1, silent=True, importance_type='split',
                                is_unbalance = False, scale_pos_weight = 1.0)

custom_pipeline = make_pipeline(
            preprocessor,
            model
            )

In [8]:
from sklearn.model_selection import StratifiedKFold
custom_pipeline.fit(train, train_label)

/home/martin/miniconda3/envs/imbalanced/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/martin/miniconda3/envs/imbalanced/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/martin/miniconda3/envs/imbalanced/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'educational-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=

In [9]:
#custom_pipeline.predict(test)
print("model score: %.3f" % custom_pipeline.score(test, test_label))

in else of transform
model score: 0.856


In [52]:
from sklearn.ensemble import GradientBoostingClassifier
clf = make_pipeline(
            preprocessor,
            GridSearchCV(
            GradientBoostingClassifier(),
            #lightgbm.LGBMClassifier(),
            param_grid={'subsample': np.linspace(0.7, 1.0, 3),
                        'max_depth': [3, 5, 9],
                        'n_estimators': [300, 400, 500],
                        'learning_rate' : 10**(np.linspace(-3, -1, 4))},
         cv=5, refit=False, scoring = 'accuracy', n_jobs=-1
)
)

In [53]:
clf.fit(train, train_label)

/home/martin/miniconda3/envs/imbalanced/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/martin/miniconda3/envs/imbalanced/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/martin/miniconda3/envs/imbalanced/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


KeyboardInterrupt: 